In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
df = pd.read_csv("HeartDiseaseTrain-Test.csv")

Start by Looking at Data

In [2]:
# use the following line to look into the first 5 records in the data
df.head()
# use the following line of code to check the info of the dataset columns and check for missing lines
df.info()
# use the following line of code to check the means/quantiles of the numeric columns
df.describe()
# use following line to check the distribution of the target (How many have heart diseases?)
df.target.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            1025 non-null   int64  
 1   sex                            1025 non-null   object 
 2   chest_pain_type                1025 non-null   object 
 3   resting_blood_pressure         1025 non-null   int64  
 4   cholestoral                    1025 non-null   int64  
 5   fasting_blood_sugar            1025 non-null   object 
 6   rest_ecg                       1025 non-null   object 
 7   Max_heart_rate                 1025 non-null   int64  
 8   exercise_induced_angina        1025 non-null   object 
 9   oldpeak                        1025 non-null   float64
 10  slope                          1025 non-null   object 
 11  vessels_colored_by_flourosopy  1025 non-null   object 
 12  thalassemia                    1025 non-null   o

1    526
0    499
Name: target, dtype: int64

Some Categories need to be encoded.  It was suggested to use the One Hot Encoder, but the supplied GUI clearly uses Label Encoding. 

Clearly the data set currently give is not the same when this course was written. Will have to struggle to get the GUI to import all the Hot Encoded values.

In [3]:
a = pd.get_dummies(df['sex'], prefix = "sex")
b = pd.get_dummies(df['chest_pain_type'], prefix = "chest_pain_type")
c = pd.get_dummies(df['fasting_blood_sugar'], prefix = "fasting_blood_sugar")
d = pd.get_dummies(df['rest_ecg'], prefix = "rest_ecg")
e = pd.get_dummies(df['exercise_induced_angina'], prefix = "exercise_induced_angina")
f = pd.get_dummies(df['slope'], prefix = "slope")
g = pd.get_dummies(df['vessels_colored_by_flourosopy'], prefix = "vessels_colored_by_flourosopy")
h = pd.get_dummies(df['thalassemia'], prefix = "thalassemia")
df = pd.concat([df,a,b,c,d,e,f,g,h], axis = 1)

df = df.drop(columns = ['sex', 'chest_pain_type', 'fasting_blood_sugar', 'rest_ecg', 'exercise_induced_angina', 'slope', 'vessels_colored_by_flourosopy', 'thalassemia'])

print(h)
print(df)
print(df.info())

      thalassemia_Fixed Defect  thalassemia_No  thalassemia_Normal  \
0                            0               0                   0   
1                            0               0                   0   
2                            0               0                   0   
3                            0               0                   0   
4                            1               0                   0   
...                        ...             ...                 ...   
1020                         1               0                   0   
1021                         0               0                   0   
1022                         1               0                   0   
1023                         1               0                   0   
1024                         0               0                   0   

      thalassemia_Reversable Defect  
0                                 1  
1                                 1  
2                                 1  
3      

Separate Target and Categories

In [4]:
y = df['target'].values
X = df.drop(['target'], axis = 1).values

Scale Categories

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)
print(scaled_X)
print(X)

[[-0.26843658 -0.37763552 -0.65933209 ... -0.08292302 -0.25806452
   1.22474487]
 [-0.15815703  0.4791073  -0.83386117 ... -0.08292302 -0.25806452
   1.22474487]
 [ 1.71659547  0.76468824 -1.39623266 ... -0.08292302 -0.25806452
   1.22474487]
 ...
 [-0.81983438 -1.23437834  0.56237149 ... -0.08292302 -0.25806452
  -0.81649658]
 [-0.4889957  -1.23437834  0.15513696 ... -0.08292302 -0.25806452
  -0.81649658]
 [-0.04787747 -0.66321646 -1.12474297 ... -0.08292302 -0.25806452
   1.22474487]]
[[ 52. 125. 212. ...   0.   0.   1.]
 [ 53. 140. 203. ...   0.   0.   1.]
 [ 70. 145. 174. ...   0.   0.   1.]
 ...
 [ 47. 110. 275. ...   0.   0.   0.]
 [ 50. 110. 254. ...   0.   0.   0.]
 [ 54. 120. 188. ...   0.   0.   1.]]


Test the Logistic Regressin Method

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(scaled_X,y,test_size=.3,random_state=42)
accuracies = {}

lr = LogisticRegression()
lr.fit(X_train,y_train)
acc = lr.score(X_test,y_test)*100

accuracies['Logistic Regression'] = acc
print("Test Accuracy {:.2f}%".format(acc))

Test Accuracy 80.84%


Test the KNN Method

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.3,random_state=42)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)  # n_neighbors means k
knn.fit(X_train, y_train)
prediction = knn.predict(X_test)

print("{} NN Score: {:.2f}%".format(2, knn.score(X_test, y_test)*100))

2 NN Score: 88.31%


Link the Model with the best accuracy.  Note:  The test and video suggest the KNN model will be worse, but the new data might make it better.  Perhaps due to scaling and encoding?

In [8]:
Final_Model = knn

      age  resting_blood_pressure  cholestoral  Max_heart_rate  oldpeak  \
0      52                     125          212             168      1.0   
1      53                     140          203             155      3.1   
2      70                     145          174             125      2.6   
3      61                     148          203             161      0.0   
4      62                     138          294             106      1.9   
...   ...                     ...          ...             ...      ...   
1020   59                     140          221             164      0.0   
1021   60                     125          258             141      2.8   
1022   47                     110          275             118      1.0   
1023   50                     110          254             159      0.0   
1024   54                     120          188             113      1.4   

      target  sex_Female  sex_Male  chest_pain_type_Asymptomatic  \
0          0           0       

Add the GUI functionality from website.  Had to modify the the predict funtion quite a bit to handle 30 data elements now that the data needed to be Encoded.

In [ ]:
def check_inputs():
    if age.get() == "":
        print("Age Field is Empty!!")
        Label(win,text="Age Field is Empty!!",fg="blue",bg="yellow",font = ("Calibri 10 bold")).place(x=12,y=580)


    elif rbp.get() == "":
        print("Resting Blood Pressure Field is Empty!!")
        Label(win,text="Resting Blood Pressure Field is Empty!!",fg="blue",bg="yellow",font = ("Calibri 10 bold")).place(x=12,y=580)

    elif chol.get() == "":
        print("Cholestrol Field is Empty!!")
        Label(win,text="Cholestrol Field is Empty!!",fg="blue",bg="yellow",font = ("Calibri 10 bold")).place(x=12,y=580)

    elif heart_rate.get() == "":
        print("Heart Rate Field is Empty!!")
        
        Label(win,text="Heart Rate Field is Empty!!",fg="blue",bg="yellow",font = ("Calibri 10 bold")).place(x=12,y=580)
    elif peak.get() == "":
        print("Depression By Exercise Field is Empty!!")
        Label(win,text="Depression By Exercise Field is Empty!!",fg="blue",bg="yellow",font = ("Calibri 10 bold")).place(x=12,y=580)

    # Doesn't check that True False fields, and drop down menus have been set.  But this is what was given
        
    else:
        predict()


def predict():
    # Add additional Dictionary for new text in .csv
    gender_dict = {"Male":1, "Female":0}
    fbs_dict = {"True":1, "False":0}
    ecg_dict = {"ST-T wave abnormality":2, "Left ventricular hypertrophy":1, "Normal":0}
    eia_dict = {"True":1, "False":0}
    cp_dict = {"1: typical angina":3,"2: atypical angina":1,"3: non-anginal pain":2,"4: asymptomatic":0}
    thal_dict = {"0: No Test":0,"1: Fixed Defect":1,"2: Normal Flow":2,"3: Reversible Defect":3}
    slope_dict = {"0: Flat":0,"1: Upsloping":1,"2: Downsloping":2,}
    Pred_dict = {0:"Prediction: No Heart Disease Detected", 1:"Prediction: Heart Disease Deteced\nYou should consult with your Doctor!"}
        
    # Set the data in order based on the df.info()
    data = [float(age.get()),float(rbp.get()), float(chol.get()),float(heart_rate.get()), float(peak.get())]
    
    #For newly encoded fields, have to enter a 1 into the correct column based on the df.info()
    #Gender
    sex = [0,0]
    if( str(radio.get()) == 0 ):
        sex[0] = 1
    else:
        sex[1] = 1
    
    data.extend(sex)
    
    #Chest Pains
    cp = [0,0,0,0]
    if( cp_dict[str(variable.get())] == 0 ):
        cp[0] = 1
    elif( cp_dict[str(variable.get())] == 1):
        cp[1] = 1
    elif( cp_dict[str(variable.get())] == 2):
        cp[2] = 1
    elif( cp_dict[str(variable.get())] == 3):
        cp[3] = 1
    
    data.extend(cp)
        
    #Fasting Blood Sugar
    fbs = [0,0]
    if( fbs_dict[str(radio_fbs.get())] == 0 ):
        fbs[0] = 1
    else:
        fbs[1] = 1
                
    data.extend(fbs)

    #Resting ECG
    ecg = [0,0,0]
    if( ecg_dict[str(variable_ecg.get())] == 1 ):
        ecg[0] = 1
    elif( ecg_dict[str(variable_ecg.get())] == 0):
        ecg[1] = 1
    elif( ecg_dict[str(variable_ecg.get())] == 2):
        ecg[2] = 1
                
    data.extend(ecg)
    
    #EIA
    eia = [0,0]
    if( eia_dict[str(radio_eia.get())] == 0 ):
        eia[0] = 1
    else:
        eia[1] = 1
                
    data.extend(eia)
    
    #Slope
    slope = [0,0,0]
    if( slope_dict[str(variable_slope.get())] == 2 ):
        slope[0] = 1
    elif( slope_dict[str(variable_slope.get())] == 0):
        slope[1] = 1
    elif( slope_dict[str(variable_slope.get())] == 1):
        slope[2] = 1
                
    data.extend(slope)
    
    #N_Vessels
    nVessels = [0,0,0,0,0]
    if( int(str(variable_n_vessels.get())) == 4 ):
        nVessels[0] = 1
    elif( int(str(variable_n_vessels.get())) == 1):
        nVessels[1] = 1
    elif( int(str(variable_n_vessels.get())) == 3):
        nVessels[2] = 1
    elif( int(str(variable_n_vessels.get())) == 2):
        nVessels[3] = 1
    elif( int(str(variable_n_vessels.get())) == 0):
        nVessels[4] = 1
         
    data.extend(nVessels)
         
    #Thal
    thal = [0,0,0,0]
    if( thal_dict[str(variable_thal.get())] == 1 ):
        thal[0] = 1
    elif( thal_dict[str(variable_thal.get())] == 0):
        thal[1] = 1
    elif( thal_dict[str(variable_thal.get())] == 2):
        thal[2] = 1
    elif( thal_dict[str(variable_thal.get())] == 3):
        thal[3] = 1
    
    data.extend(thal)
        
    prediction = Final_Model.predict(np.array(data).reshape(1,30))
    pred_label = Pred_dict[prediction.tolist()[0]]
    Label(win,text=pred_label,fg="blue",bg="yellow",font = ("Calibri 10 bold")).place(x=12,y=580)



def reset():
    age.set("")
    rbp.set("")
    chol.set("")
    heart_rate.set("")
    peak.set("")


import re
from tkinter import *
import tkinter as tk


win =  Tk()

win.geometry("450x600")
win.configure(background="#Eaedee")
win.title("Heart Disease Classifier")
#win.iconbitmap('icon.ico')

title = Label(win, text="Heart Disease Classifier", bg="#2583be", width="300", height="2", fg="white", font = ("Arial 20 italic")).pack()

age = Label(win, text="Age in Years", bg="#Eaedee", font=("Verdana 12")).place(x=12, y=65)

rbp = Label(win, text="Resting Blood Pressure ", bg="#Eaedee", font=("Verdana 12")).place(x=12, y=105)

chol = Label(win, text="Cholestrol mg/dl ", bg="#Eaedee", font=("Verdana 12")).place(x=12, y=145)

heart_rate = Label(win, text="Maximum Heart Rate ", bg="#Eaedee", font=("Verdana 12")).place(x=12, y=185)

peak = Label(win, text="Depression By Exercise ",bg="#Eaedee",font = ("Verdana 12")).place(x=12,y=225)


  
Gender = Label(win, text="Gender ",bg="#Eaedee",font = ("Verdana 12")).place(x=12,y=265)

radio = StringVar()
Male = Radiobutton(win, text="Male",bg="#Eaedee",variable=radio,value="Male",font = ("Verdana 12")).place(x=160,y=265)
Female = Radiobutton(win, text="Female",bg="#Eaedee",variable=radio,value="Female",font = ("Verdana 12")).place(x=260,y=265)

FBS = Label(win, text="Fasting Blood Pressure ",bg="#Eaedee",font = ("Verdana 12")).place(x=12,y=285)

radio_fbs = StringVar()
Male = Radiobutton(win, text="True",bg="#Eaedee",variable=radio_fbs,value="True",font = ("Verdana 12")).place(x=160,y=285)
Female = Radiobutton(win, text="False",bg="#Eaedee",variable=radio_fbs,value="False",font = ("Verdana 12")).place(x=260,y=285)

EIA = Label(win, text="Exercise Induced Angina",bg="#Eaedee",font = ("Verdana 12")).place(x=12,y=305)

radio_eia = StringVar()
Male = Radiobutton(win, text="True",bg="#Eaedee",variable=radio_eia,value="True",font = ("Verdana 12")).place(x=160,y=305)
Female = Radiobutton(win, text="False",bg="#Eaedee",variable=radio_eia,value="False",font = ("Verdana 12")).place(x=260,y=305)


cp = Label(win,text="Chest Pain ",bg="#Eaedee",font = ("Verdana 12")).place(x=12,y=345)
variable = StringVar(win)
variable.set("CP")
w = OptionMenu(win, variable, "1: typical angina","2: atypical angina","3: non-anginal pain","4: asymptomatic")
w.place(x=140,y=345)

ecg = Label(win,text="Resting ECG ",bg="#Eaedee",font = ("Verdana 12")).place(x=12,y=385)
variable_ecg = StringVar(win)
variable_ecg.set("ECG")
w_ecg = OptionMenu(win, variable_ecg, "Normal","Left ventricular hypertrophy","ST-T wave abnormality")
w_ecg.place(x=140,y=385)


exer_slope = Label(win,text="Slope of Exercise ",bg="#Eaedee",font = ("Verdana 12")).place(x=12,y=425)
variable_slope = StringVar(win)
variable_slope.set("Slope")
w_slope = OptionMenu(win, variable_slope, "0: Flat","1: Upsloping","2: Downsloping")
w_slope.place(x=140,y=425)

thal_label = Label(win,text="Thallium Stress ",bg="#Eaedee",font = ("Verdana 12")).place(x=12,y=465)
variable_thal = StringVar(win)
variable_thal.set("Thal")
w_thal = OptionMenu(win, variable_thal, "0: No Test","1: Fixed Defect","2: Normal Flow","3: Reversible Defect")
w_thal.place(x=140,y=465)


n_vessels = Label(win,text="Number Vessels ",bg="#Eaedee",font = ("Verdana 12")).place(x=12,y=505)
variable_n_vessels = StringVar(win)
variable_n_vessels.set("N_Vessels")
w_n_vessels = OptionMenu(win, variable_n_vessels, "0","1","2","3","4")
w_n_vessels.place(x=140,y=505)


age = StringVar()
rbp = StringVar()
chol = StringVar()
heart_rate = StringVar()
peak  = StringVar()
Gender = StringVar()
FBS  = StringVar()
EIA  = StringVar()
cp  = StringVar()
ecg  = StringVar()
exer_slope  = StringVar()
thal_label  = StringVar()
n_vessels  = StringVar()

entry_age = Entry(win,textvariable = age,width=30)
entry_age.place(x=150,y=65)

entry_rbp = Entry(win,textvariable = rbp,width=30)
entry_rbp.place(x=150,y=105)

entry_chol = Entry(win,textvariable = chol,width=30)
entry_chol.place(x=150,y=145)

entry_heart_rate = Entry(win, textvariable = heart_rate,width=30)
entry_heart_rate.place(x=150,y=185)

entry_peak = Entry(win,textvariable = peak,width=30)
entry_peak.place(x=150,y=225)

reset = Button(win, text="Reset", width="12",height="1",activebackground="red",command=reset, bg="Pink",font = ("Calibri 12 ")).place(x=24, y=540)
submit = Button(win, text="Classify", width="12",height="1",activebackground="violet", bg="Pink",command=check_inputs,font = ("Calibri 12 ")).place(x=240, y=540)


win.mainloop()